In [9]:
from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True, reshape=False)

import tensorflow as tf
import matplotlib.pyplot as plt 
import numpy as np
import math

learning_rate = 0.001
training_epochs = 20
batch_size = 100


X = tf.placeholder(tf.float32, shape=[None, 28, 28, 1], name="input")

Y_ = tf.placeholder(tf.float32, shape=[None, 10], name="labels")

step = tf.placeholder(tf.int32)



def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])


h_conv1 = tf.nn.relu(conv2d(X, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])


h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

Ylogits = tf.matmul(h_fc1, W_fc2) + b_fc2
Y = tf.nn.softmax(Ylogits)


loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=Ylogits, labels=Y_)
loss = tf.reduce_mean(loss)*100


# training step, the learning rate is a placeholder
# the learning rate is: # 0.0001 + 0.003 * (1/e)^(step/2000)), i.e. exponential decay from 0.003->0.0001
lr = 0.0001 +  tf.train.exponential_decay(0.003, step, 2000, 1/math.e)
train_step = tf.train.AdamOptimizer(lr).minimize(loss)



correct_prediction = tf.equal(tf.argmax(Y,1), tf.argmax(Y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

print("\n")

display_step = 1
n_iterations = []  
n_loss = []
n_loss_test = []
n_train_accuracy = []
n_test_accuracy = []

with tf.Session() as sess:

    
    sess.run(init)
    
    total_steps = 0;
    
    for epoch in range(training_epochs):
        avg_cost = 0.
        avg_accuracy = 0.
        avg_accuracy_test = 0.0
        total_batch = int(mnist.train.num_examples/batch_size)
        
        for i in range(total_batch):
            total_steps = total_steps +1
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            
            
            #Train
            sess.run(train_step, feed_dict={X: batch_xs, Y_: batch_ys, step: total_steps})
           
            #Evaluate
            train_accuracy, c = sess.run([accuracy,loss],feed_dict={X: batch_xs, Y_: batch_ys})
            test_accuracy, c_test = sess.run([accuracy,loss],feed_dict={X: mnist.test.images, Y_: mnist.test.labels})
          
            avg_cost += c / total_batch
            avg_accuracy += train_accuracy /total_batch
            avg_accuracy_test += test_accuracy /total_batch
            
            n_iterations.append(total_steps)
            n_loss.append(c)
            n_loss_test.append(c_test)
            n_train_accuracy.append(train_accuracy)
            n_test_accuracy.append(test_accuracy)
        
        
        if (epoch+1) % display_step == 0 or epoch == 0:
            print("Epoch:", '%04d' % (epoch+1), " Cost=", "{:.9f}".format(avg_cost), " Train_Accuracy=", "{:.9f}".format(avg_accuracy)," Test_Accuracy=", "{:.9f}".format(avg_accuracy_test))
            

    print("Training Finished!")

    
    
    final_test_accuracy = sess.run(accuracy,feed_dict={X: mnist.test.images, Y_: mnist.test.labels})
    print("\nTest_Accuracy:", final_test_accuracy)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


Epoch: 0001  Cost= 35.876851542  Train_Accuracy= 0.921363641  Test_Accuracy= 0.920525273
Epoch: 0002  Cost= 5.231365057  Train_Accuracy= 0.984363647  Test_Accuracy= 0.979543636
Epoch: 0003  Cost= 3.327835022  Train_Accuracy= 0.990127281  Test_Accuracy= 0.983641636
Epoch: 0004  Cost= 2.412056592  Train_Accuracy= 0.992636370  Test_Accuracy= 0.985661455
Epoch: 0005  Cost= 1.704739505  Train_Accuracy= 0.995163641  Test_Accuracy= 0.986325091
Epoch: 0006  Cost= 1.265047532  Train_Accuracy= 0.996600003  Test_Accuracy= 0.987440000
Epoch: 0007  Cost= 0.955304487  Train_Accuracy= 0.997509093  Test_Accuracy= 0.987713820
Epoch: 0008  Cost= 0.683852998  Train_Accuracy= 0.998363638  Test_Accuracy= 0.988266363
Epoch: 0009  Cost= 0.475258923  Train_Accuracy= 0.999127274  Test_Accuracy= 0.988146001
Epoch: 0010  Co